# MRN Normalization

1.  Loading data into a gene expression matrix.
2.  Munging data
3.  MRN Normalization
4.  sanity checks between clones 

"Trimmed Mean of M-values" (TMM) normalization, published by [Robinson and Oshlack](https://www.frontiersin.org/articles/10.3389/fgene.2016.00164/full#B16) is a widely used method of normalizing gene expression in scRNA data.  A variant called MRN (Median Ratio Normalization) is described by [Maza et al.](https://www.tandfonline.com/doi/full/10.4161/cib.25849), and may perform slightly better than TMM.  We carry out MRN normalization on the P9855 RNA data, and use the results in a quick machine learning application.

In [2]:
#Import packages.  Put plots "inline" in the notebook.  

import numpy as np  # For numerical computations.
import pandas as pd  # Pandas for data analysis.
import matplotlib.pyplot as plt  # For basic plotting.
import seaborn as sns # For pretty visualization in Seaborn.  See https://seaborn.pydata.org/

import os # Working with file directories, etc.

from IPython.display import display # Pretty display of data frames.

# Put plots inline rather than in a pop-up.
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #got annoyed of these red boxes

Go to section 2, if you have already loaded the data and have the pickles.

# 1.  Loading the data

Start at section 2, if the raw data has already been processed and pickled.

In this section, we load the gene expression data and some metadata.  The sequencing data is stored in a 'merged_gene_counts.txt'.  The metadata is stored in a separate file called 'meta_data_marty_inVitro_feb8.csv'.  We use the metadata to select bulk-cells from  experiment P9855.  The first function loads the metadata.

In [ ]:
def get_meta(experiment, filename = 'meta_data_marty_inVitro_feb8.csv', report=True, bulks = False):
    df = pd.read_csv(filename, sep=',',
                     index_col=2, header=0, low_memory=False)
    df = df[ df['Project_ID'] == experiment] # Only cells from the experiment.
    if bulks:
        df = df[ df['Number_Of_Cells'] >= 25] # Include only bulks.
    else:
        df = df[ df['Number_Of_Cells'] == 1] # Get rid of bulks.
    if report:
        if bulks:
            print('{} bulks found in experiment {}'.format(len(df), experiment))
        else:
            print('{} single cells found in experiment {}'.format(len(df), experiment))
        clones = df['Clone_ID'].unique()
        print('{} Clones: {}'.format(len(clones), ', '.join(clones)))
        print('The first five rows of the dataframe are below')
        display(df[:5])
    return df

In [ ]:
meta_df = get_meta('P9855', filename = 'meta_data_marty_inVitro_feb8.csv')

In [ ]:
meta_bulks = get_meta('P9855', filename = 'meta_data_marty_inVitro_feb8.csv', bulks=True)

In [ ]:
meta_bulks.drop(meta_bulks.index[[range(69,99)]], inplace=True) # drop atac data... and c12 while we are cutting
                                                                # the tail off of the dataframe 
meta_bulks.drop(['In_Vivo_Clone_ID'],axis=1, inplace=True)
meta_bulks.sort_index().head()

In [ ]:
meta_bulks.shape

In [ ]:
meta_bulks['Clone_ID'].unique()

The sequencing data (in merged_gene_counts.txt) contains a separate row for each  gene.  The name of each column contains list of filenames (I'm guessing input for star), we will clean up column names.
But first, to take out bulks that are negatives or otherwise messed up... I'm doing this the brute way by excluding by a list. *If I were clever I would look at rows as triplets and make sure all three have 25 cells.*

In [ ]:
exclude_samples = ['P9855_2016', 'P9855_2017', 'P9855_2018', 'P9855_2043', 'P9855_2044', 'P9855_2045', 'P9855_2046', 'P9855_2047', 'P9855_2048', 'P9855_2055', 'P9855_2056', 'P9855_2057', 'P9855_2082', 'P9855_2083', 'P9855_2084']

In [ ]:
read_merged_genes = pd.read_csv('merged_gene_counts.txt', sep='\t', index_col=0)
read_merged_genes.columns = read_merged_genes.columns.str[:10] # 10 characters in P9855_20**
read_merged_genes.drop(['gene_name'],axis=1, inplace=True)
read_merged_genes = read_merged_genes.transpose() # tanspose dataframe for later analysis infrastructure
read_merged_genes.drop(list(read_merged_genes.index & exclude_samples), inplace=True)
read_merged_genes.sort_index().head()

In [ ]:
read_merged_genes.shape # great, the sizes for the meta and the rawreads have equal index sizes

In [ ]:
read_merged_genes.to_pickle('P9855_rawreads.pkl') # Save file as a pickle.
meta_bulks.to_pickle('P9855_meta.pkl') # Pickle the metadata too.

The next function loads an entire *list* of cells, and places their gene expression data into the rows of a matrix.  The rows are indexed by the cell names, and the columns by genes.  The data is the gene expression, as raw number of reads.  This may take a little while, so we give progress updates every 10 cells.

# 2.  Munging data

Start here if you have the pickles!  We filter the data a bit, before normalization downstream.

In [ ]:
bulks_raw = pd.read_pickle('P9855_rawreads.pkl') # Load bulks expression matrix from a pickle.
meta_df = pd.read_pickle('P9855_meta.pkl') # Load metadata from a pickle.

In [ ]:
genes = list(bulks_raw.columns)  # The names of the genes. 
bulks = list(bulks_raw.index) # The names of the bulks.
clones = sorted(list(meta_df.Clone_ID.unique())) # The names of the clones.

## Removing TCRs and rarely-expressed genes

T cells have special genetically rearranged receptors called TCRs.  These are made of segments called TRBV9, TRBJ2-4, TRAV12-2, TRAJ14, etc.  Bascally any gene that is called these letters followed by a number -- TRBV, TRBJ, TRAV, TRAJ -- is part of this receptor and they are defined as being clonal.  Therefore we exclude these genes since we want to find more interesting similarities within clonal populations.

The following loads a list of genes to be excluded from the data for later analysis.  The excluded genes should be given in a csv file with *one* column.  No row labels should be given.  The first row should be a descriptive header, like "Genes to exclude."

In [ ]:
exc_filename = 'TRgenes.csv'  # CHANGE this if needed.  I added TRAC and TRDV3 as requested.
exc_df = pd.read_csv(exc_filename, sep=',', header=0)
exclude_genes = exc_df.iloc[:,0].tolist()

Next, we switch the labels from the hgnc name to the ensemble ID because there were two instances with the same hgnc name, however, they have unique ensemble IDs.

In [ ]:
ensembl_hgnc_meta_filename = 'All_Gene_Info_Metadata.csv'
ensembl_hgnc_meta_df = pd.read_csv(ensembl_hgnc_meta_filename, sep=',', header=0)
ensembl_genes = pd.Series(ensembl_hgnc_meta_df.ensembl_gene_id.values,index=ensembl_hgnc_meta_df.hgnc_symbol).to_dict()
exclude_ensembl = [ensemblID for hgnc,ensemblID in ensembl_genes.items() if hgnc in exclude_genes]

In [ ]:
def get_relevant(gf, eg, prevalence=0.05, threshold = 10): # gf = expression matrix, eg = list of genes to exclude
    '''
    Outputs True if the gene is relevant for analysis.  We throw out excluded genes.
    By default, we take genes that are found in at least 5% of all cells at a level of
    10 counts or more.
    '''
    nonzero_count = (gf > threshold).sum(axis=0)
    nonzero_proportion = nonzero_count / len(gf)
    return [gene for gene in gf.columns if
           (gene not in eg) & 
           (nonzero_proportion[gene] > prevalence)]

In [ ]:

genes_relevant = get_relevant(bulks_raw, exclude_ensembl) # takes like a min


In [ ]:
print("{} bulks are measured, from {} to {}.".format(len(bulks), bulks[0], bulks[-1]))
print("{} genes are measured, from {} to {}.".format(len(genes),genes[0],genes[-1]))
genes_excluded = [gene for gene in exclude_ensembl if gene in genes]
print("{} TCR genes were excluded, from {} to {}.".format(len(genes_excluded), genes_excluded[0], genes_excluded[-1]))
print("{} genes are considered relevant, from {} to {}.".format(len(genes_relevant), genes_relevant[0], genes_relevant[-1]))

## Removing low count libraries

Some triplets (sets of clones) samples expressed too few genes. We make "violin-plots" giving the number of genes expressed by each cell, sorted by clonality.

In [ ]:
def nRead(ge, md, cutoff = 0, plot=True): # ge = gene expression, md = meta data
    nR = ge.apply(lambda row: sum(row > cutoff), axis=1) # cutoff = number of genes expressed?
    nR.name = 'num_reads'
    clonalities = md.Clone_ID
    nRead_df = pd.concat([nR, clonalities], axis=1)
    if plot:
        fig = plt.subplots(figsize=(18,8))
        sns.violinplot(x="Clone_ID", y="num_reads", inner='quartiles', data=nRead_df)
        sns.swarmplot(x="Clone_ID", y="num_reads", color="white", size=3, data=nRead_df);
    return nRead_df

In [ ]:
nR = nRead(bulks_raw, meta_df)

Next, we drop samples that have low read counts below two standard deviations from the average total raw counts across samples.

In [ ]:
nR_sum = bulks_raw.apply(np.sum, axis=1) # total raw counts per sample 
nR_mean = nR_sum.mean() # across samples, the avg total raw counts
nR_std = nR_sum.std() # across samples, the std total raw counts
nR_lowcut = (nR_mean - 2*nR_std)
bulks_good = [b for b in bulks if 
              (bulks_raw.sum(axis=1).loc[b] >= nR_lowcut)]
print('{} bulks remaining after {} poor libraries removed.'.format(len(bulks_good), len(bulks) - len(bulks_good)))

In [ ]:
nR = nRead(bulks_raw.loc[bulks_good], meta_df.loc[bulks_good]) # Post-trimming violin-plot.

In [ ]:
#ignore this block
def get_bulks(ge, md, counts_threshold = 2000): # ge = gene expression, md = meta data
    '''
    this function adds up all the raw counts per sample, if any sample falls below the set threshold
    the entire clone's data (the triplet of samples) will be dropped.
    '''
    sample_counts = ge.sort_index().sum(axis=1)

    for sample in range(0, len(sample_counts),3):
        triplet = sample_counts.iloc[sample: sample + 3].values
        print(triplet,'a triplet')
        #print(triplet.values,'two')
        if np.all(triplet) > counts_threshold:
            print('valid triplet', triplet)
#ignore this block

#  3.  MRN Normalization.

Here we implement MRN Normalization on the gene expression data, closely following the convenient outline in Section 3.2 of [Maza](https://www.frontiersin.org/articles/10.3389/fgene.2016.00164/full).  We begin by putting our filtered data into a dataframe.  The dataframe `EM` (for "expression matrix") contains the gene expressions for the cells, filtered as above.

In [ ]:
EM = bulks_raw[genes_relevant].loc[bulks_good]

In [ ]:
EM.sort_index().head()

In [ ]:
EM.shape

## Step I:  Prenormalization by library size.

In Maza's article, $X_{gkr}$ stands for the raw count (number of reads) of gene $g$, for a cell number $r$ among clone $k$.  This information is contained in our expression matrix `EM`.  The first step is to normalize by library size, dividing $X_{gkr}$ by the total number of reads $N_{kr}$ of cell with numbers $k$, $r$.  

We find the total number of reads for each cell by simply summing the numbers in each row of the data frame `EM`.  We don't need to worry about the separate indices $k$ and $r$ yet.  We examine the resulting "library size" below.

In [ ]:
library_size = EM.sum(axis=1) # Drop the clone column.  Sum along rows.
print(library_size.head())
library_size.describe()

The library size is about 1.7 million +/- 280,000.  Now we normalize the expression matrix by dividing every cell's raw counts by the cell's library size.  Note that `EM` is a dataframe whose rows are indexed by the cells.  `library_size` is a series (basically an array) whose rows are indexed by cells.  Numpy/pandas will divide one array by another, term by term, if they have the same size.  So it can divide *each column* of `EM` by `library size`, in a quickly-broadcasted division.  To perform this on every column, we use the `apply` method with a "lambda" function... it's the quickest method I know.

In [ ]:
Y = EM.apply(lambda column : column / library_size)
Y.sort_index().head() #does not update the Y expression matrix, just for the sake of  us looking at nicely sorted rows

Now `Y` is the dataframe with counts normalized by library size, and we pass to the next step.

## Step II:  Creation of reference sample.

A difference between TMM (used in edgeR), RLE (used in DeSeq2), and MRN, is how they create a reference sample.  In MRN, a reference sample is created by averaging the previous dataframe `Y` over cells within a single condition (clone).  We carry this out here.

Note the method-chaining in defining `Y_clonal` below.  The `assign` method tacks on a new column to `Y` for the clone_IDs.  The `groupby` method then groups cells by clonotypes.  The `apply` method then takes the means within each clonotype.  The end result is to replace the (prenormalized) gene expression for *each cell* by the averages for each clonotype.  

In [ ]:
Y_clonal = Y.assign(Clone=meta_df['Clone_ID']).groupby('Clone').apply(np.mean) # Takes a few seconds.
display(Y_clonal)

Let's take our reference sample to be the A3 clonal average.

In [ ]:
Y_ref = Y_clonal.loc['P1_A03'] # Our reference sample.  Basically an average of all cells of clonotype P1_A03.
Y_ref.describe()

## Step III:  Computation of size relative to reference sample.

Now we compute relative scaling factors for each clone, based on the median fold-changes between gene expression within that clone and the reference clone (A7, we have chosen above).  Due to dropouts (values of 0 in gene expression), we discard genes with zeros when computing fold changes.  This avoids division-by-zero problems, and it matches the implementation of [Maza et al.](https://www.tandfonline.com/doi/full/10.4161/cib.25849).  (See the R code in the supplementary information).  

In [ ]:
tau = pd.Series(index = clones)
for clone in clones: # Why not a little loop.
    numerator = Y_clonal.loc[clone]
    denominator = Y_ref
    ok_genes = [gene for gene in genes_relevant if (numerator[gene] != 0) and (denominator[gene] != 0)]
    ratios = numerator[ok_genes] / denominator[ok_genes] # The ratios.
    tau[clone] = np.median(ratios)
print(tau)

## Step IV:  Adjustment of relative scaling factors.

This step does not occur in MRN normalization.  Yay!

## Step V:  Effective library size.

In [ ]:
# intermediate series to match the clone IDs with the samples
clonalities = pd.Series(meta_df['Clone_ID'], index=library_size.index)
clonalities.sort_index().head()

In [ ]:
#intermediate series to map tau scaling values to the samples
scalar_lib_size = clonalities.map(tau)
scalar_lib_size.sort_index().head()

In [ ]:
#final series, for this step at least as the library_size values are scaled by tau values
effective_lib_size = scalar_lib_size * library_size
effective_lib_size.sort_index().head()

## Step VI:  Computation of relative normalization/size factors.

In [ ]:
geo_mean = np.exp(effective_lib_size.apply(np.log).mean()) # The geometric mean of the effective_lib_size series.
geo_mean # A single number!

In [ ]:
effective_lib_size.mean() # Compare the geometric mean (above) to the usual mean here.  They shouldn't be *too* far.

In [ ]:
relative_normalization_factor = effective_lib_size / geo_mean
relative_normalization_factor.sort_index().head()

## Step VII:  Normalization of counts.

In [ ]:
# here we normalize our raw counts by the relative normalization factor we just
# calculated in Step VI
mrn_counts = EM.div(relative_normalization_factor, axis=0)
mrn_counts.sort_index().head()

In [ ]:
mrn_counts.shape

In [ ]:
mrn_counts.to_pickle('P9855_mrn.pkl') # Pickle the counts dataframe.

In [ ]:
mrn_counts.to_csv('P9855_mrn.csv') # write the counts dataframe to a file that can be opened in excel.

# 4.  sanity checks between clones 

After preprocessing the data, one might want to quickly check how gene expression varies or lack thereof between clones.Both CD3D (ENSG00000167286) and ACTB (ENSG00000075624) just **may** be similiarly expressed between cells. Whereas, we use KLRD1 (ENSG00000134539) and IL4 (ENSG00000113520), which should both vary between clones. 

In [16]:
X = pd.read_pickle('P9855_mrn.pkl') # Load normalized data from a pickle.
X_cells = X.index
y_meta = pd.read_pickle('P9855_meta.pkl') # Load metadata from a pickle.
y_clonal = X.assign(Clone=y_meta['Clone_ID']).groupby('Clone').apply(np.mean) # Takes a minute.

In [19]:
check_GOIs = y_clonal[['ENSG00000167286','ENSG00000075624','ENSG00000134539','ENSG00000113520']]
check_GOIs #

Geneid,ENSG00000167286,ENSG00000075624,ENSG00000134539,ENSG00000113520
Clone,,,,
P1_A03,6345.318780,4445.952739,252.247889,0.000000
P1_A05,4919.983706,6039.982829,63.355645,60.929846
P1_A08,5117.387436,5021.683918,687.836586,0.000000
P1_A12,5301.474785,6808.072023,55.307823,0.000000
P1_B10,4070.314569,6079.636062,112.577324,10.695181
P1_C11,6884.670186,4983.576678,1006.945061,124.851286
P1_D09,5121.110087,13308.868565,0.000000,13.563975
P1_D12,5856.853798,4869.036502,920.274802,0.000000
P1_F02,4964.108172,3346.955712,0.000000,0.000000
